In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model

In [3]:
properties = pd.read_csv('properties_2016.csv')
train = pd.read_csv('train_2016.csv')
sampleSub = pd.read_csv('sample_submission.csv')

/Users/adit/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [9]:
properties.tail()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
2985212,168176230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,14273630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,168040630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,168040830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985216,168040430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
sampleSub.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [2]:
p = pd.read_csv('preds.csv')

In [3]:
p.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147.0,0.004256,0.004256,0.004256,0.004256,0.004256,0.004256
1,10759547.0,0.004097,0.004097,0.004097,0.004097,0.004097,0.004097
2,10843547.0,0.313878,0.313878,0.313878,0.313878,0.313878,0.313878
3,10859147.0,0.014173,0.014173,0.014173,0.014173,0.014173,0.014173
4,10879947.0,0.001112,0.001112,0.001112,0.001112,0.001112,0.001112


In [24]:
cleanedPropertyData = pd.np.array(properties)
cleanedPropertyData[0][0]

10754147

In [23]:
(cleanedPropertyData[0][0])

10754147

In [150]:
properties.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,regionidcounty,storytypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
count,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,...,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06
mean,1.332586e+07,1.253134e+00,7.000411e+00,6.468833e+02,2.208340e+00,3.088608e+00,3.998841e+00,6.210886e+00,2.286339e+00,5.726887e-03,...,3.977168e-01,5.440141e-04,1.120013e+00,3.198034e+02,2.782966e+02,1.964262e+03,3.202096e-01,1.708836e+05,2.524780e+05,5.377607e+03
std,7.909966e+06,1.693104e+00,1.101467e-01,1.257848e+01,1.075760e+00,1.273421e+00,3.716363e-02,1.566111e+00,9.807784e-01,7.545920e-02,...,6.235725e-01,2.331777e-02,1.994175e+00,3.824130e+01,1.100762e+01,2.320484e+01,6.423593e-01,3.983485e+05,4.399356e+05,9.134915e+03
min,1.071172e+07,1.000000e+00,2.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+01,1.000000e+01,1.801000e+03,0.000000e+00,1.000000e+00,1.000000e+00,1.340000e+00
25%,1.164371e+07,1.000000e+00,7.000000e+00,6.468833e+02,2.000000e+00,2.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,3.198034e+02,2.782966e+02,1.950000e+03,0.000000e+00,7.564100e+04,7.710000e+04,2.478050e+03
50%,1.254509e+07,1.000000e+00,7.000000e+00,6.468833e+02,2.000000e+00,3.000000e+00,4.000000e+00,7.000000e+00,2.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,3.198034e+02,2.782966e+02,1.964000e+03,0.000000e+00,1.245920e+05,1.718430e+05,4.027760e+03
75%,1.409712e+07,1.000000e+00,7.000000e+00,6.468833e+02,3.000000e+00,4.000000e+00,4.000000e+00,7.000000e+00,3.000000e+00,0.000000e+00,...,1.000000e+00,0.000000e+00,1.000000e+00,3.198034e+02,2.782966e+02,1.981000e+03,0.000000e+00,1.947780e+05,3.022000e+05,6.166200e+03
max,1.696019e+08,1.300000e+01,2.700000e+01,8.516000e+03,2.000000e+01,2.000000e+01,5.000000e+00,1.200000e+01,2.000000e+01,1.000000e+00,...,2.000000e+00,1.000000e+00,9.970000e+02,7.983000e+03,6.141000e+03,2.015000e+03,4.100000e+01,2.514860e+08,9.024622e+07,3.458861e+06


In [141]:
xTrain.shape

(90811, 37)

In [120]:
properties[properties['parcelid'] == 10754147]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,regionidcounty,storytypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
0,10754147,1.0,7.0,646.883292,0.0,0.0,4.0,7.0,2.0,0.0,...,0.0,0.0,1.0,319.803397,278.296562,1964.261641,0.0,170883.577166,9.0,5377.607139


In [151]:
numTrainExamples = train.shape[0]
numFeatures = properties.shape[1]
xTrain = np.zeros([numTrainExamples, numFeatures])
yTrain = np.zeros([numTrainExamples])

propertiesIds = properties['parcelid']
rowCounter = 0
for index, row in train.iterrows():
    xTrain[rowCounter] = properties[properties['parcelid'] == row['parcelid']]
    yTrain[rowCounter] = row['logerror']
    rowCounter = rowCounter + 1
    if index == 10:
        break

In [11]:
type(properties.loc[0, 'parcelid'])

numpy.int64

In [163]:
model = linear_model.LinearRegression()
model.fit(xTrain, yTrain)
preds = model.predict(properties)

In [15]:
cleanedPropertyData = pd.np.array(properties)
type((cleanedPropertyData[0][0]))

KeyboardInterrupt: 

In [19]:
(cleanedPropertyData[0][0])

10754147

In [166]:
numTestExamples = properties.shape[0]
numPredictionColumns = 7
predictions = np.zeros([numTestExamples, numPredictionColumns])
for index, pred in enumerate(preds):
    predictions[index][0] = properties.loc[index, 'parcelid']
    predictions[index][1:7] = pred
    break
predictions[0]

array([  1.07541470e+07,   5.47929060e-01,   5.47929060e-01,
         5.47929060e-01,   5.47929060e-01,   5.47929060e-01,
         5.47929060e-01])

In [139]:
sampleSub.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [167]:
import csv
firstRow = [['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712']]
with open("result.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(firstRow)
    writer.writerows(predictions)

# Filling in NA Values

In [149]:
airConditionMode = properties['airconditioningtypeid'].value_counts().argmax()
properties['airconditioningtypeid'] = properties['airconditioningtypeid'].fillna(airConditionMode)

architectureMode = properties['architecturalstyletypeid'].value_counts().argmax()
properties['architecturalstyletypeid'] = properties['architecturalstyletypeid'].fillna(architectureMode)

basementSqFeetAverage = properties['basementsqft'].mean()
properties['basementsqft'] = properties['basementsqft'].fillna(basementSqFeetAverage)

bathroomCntMode = properties['bathroomcnt'].value_counts().argmax()
properties['bathroomcnt'] = properties['bathroomcnt'].fillna(bathroomCntMode)

bedroomCntMode = properties['bedroomcnt'].value_counts().argmax()
properties['bedroomcnt'] = properties['bedroomcnt'].fillna(bedroomCntMode)

buildingClassType = properties['buildingclasstypeid'].value_counts().argmax()
properties['buildingclasstypeid'] = properties['buildingclasstypeid'].fillna(buildingClassType)

buildingQualityType = properties['buildingqualitytypeid'].value_counts().argmax()
properties['buildingqualitytypeid'] = properties['buildingqualitytypeid'].fillna(buildingQualityType)

calculatedBathnBedroom = properties['calculatedbathnbr'].value_counts().argmax()
properties['calculatedbathnbr'] = properties['calculatedbathnbr'].fillna(calculatedBathnBedroom)

# Making deck type a binary label
properties['decktypeid'] = properties['decktypeid'].fillna(0) 
properties['decktypeid'] = properties['decktypeid'].replace(66,1)

floor1SqFeetAverage = properties['finishedfloor1squarefeet'].mean()
properties['finishedfloor1squarefeet'] = properties['finishedfloor1squarefeet'].fillna(floor1SqFeetAverage)

calculatedSqFeetAverage = properties['calculatedfinishedsquarefeet'].mean()
properties['calculatedfinishedsquarefeet'] = properties['calculatedfinishedsquarefeet'].fillna(calculatedSqFeetAverage)

finishedSqFeet6 = properties['finishedsquarefeet6'].mean()
properties['finishedsquarefeet6'] = properties['finishedsquarefeet6'].fillna(finishedSqFeet6)

finishedSqFeet12 = properties['finishedsquarefeet12'].mean()
properties['finishedsquarefeet12'] = properties['finishedsquarefeet12'].fillna(finishedSqFeet12)

finishedSqFeet13 = properties['finishedsquarefeet13'].mean()
properties['finishedsquarefeet13'] = properties['finishedsquarefeet13'].fillna(finishedSqFeet13)

finishedSqFeet15 = properties['finishedsquarefeet15'].mean()
properties['finishedsquarefeet15'] = properties['finishedsquarefeet15'].fillna(finishedSqFeet15)

finishedSqFeet50 = properties['finishedsquarefeet50'].mean()
properties['finishedsquarefeet50'] = properties['finishedsquarefeet50'].fillna(finishedSqFeet50)

fips = properties['fips'].value_counts().argmax()
properties['fips'] = properties['fips'].fillna(fips)

# Making fireplace count\ a binary label
properties['fireplacecnt'] = properties['fireplacecnt'].replace([2,3,4,5,6,7,8,9],1)
properties['fireplacecnt'] = properties['fireplacecnt'].fillna(0) 

fullCntBathMode = properties['fullbathcnt'].value_counts().argmax()
properties['fullbathcnt'] = properties['fullbathcnt'].fillna(fullCntBathMode)

garageCntMode = properties['garagecarcnt'].value_counts().argmax()
properties['garagecarcnt'] = properties['garagecarcnt'].fillna(garageCntMode)

garageSqFeetMode = properties['garagetotalsqft'].value_counts().argmax()
properties['garagetotalsqft'] = properties['garagetotalsqft'].fillna(garageSqFeetMode)

# Making hot tub a binary label
properties['hashottuborspa'] = properties['hashottuborspa'].replace(True,1)
properties['hashottuborspa'] = properties['hashottuborspa'].fillna(0)

heatingMode = properties['heatingorsystemtypeid'].value_counts().argmax()
properties['heatingorsystemtypeid'] = properties['heatingorsystemtypeid'].fillna(heatingMode)

properties = properties.drop('latitude', axis=1)
properties = properties.drop('longitude', axis=1)

lotSizeMode = properties['lotsizesquarefeet'].value_counts().argmax()
properties['lotsizesquarefeet'] = properties['lotsizesquarefeet'].fillna(lotSizeMode)

# Making pool a binary label
properties['poolcnt'] = properties['poolcnt'].fillna(0)

properties['poolsizesum'] = properties['poolsizesum'].fillna(0)

# These properties show through with the previous features
properties = properties.drop('pooltypeid10', axis=1)
properties = properties.drop('pooltypeid2', axis=1)
properties = properties.drop('pooltypeid7', axis=1)

# Why would these even impact the price (but idk, maybe they're important)?
properties = properties.drop('propertycountylandusecode', axis=1)
properties = properties.drop('propertylandusetypeid', axis=1)
properties = properties.drop('propertyzoningdesc', axis=1)
properties = properties.drop('rawcensustractandblock', axis=1)
properties = properties.drop('censustractandblock', axis=1)

properties['regionidcounty'] = properties['regionidcounty'].replace([3101, 1286, 2061],[0,1,2])
properties['regionidcounty'] = properties['regionidcounty'].fillna(0) 

# No idea how to handle these features
properties = properties.drop('regionidcity', axis=1)
properties = properties.drop('regionidzip', axis=1)
properties = properties.drop('regionidneighborhood', axis=1)

# Don't bedroom and bathroom counts already do this?
properties = properties.drop('roomcnt', axis=1)
properties = properties.drop('threequarterbathnbr', axis=1)

# Making deck type a binary label
properties['storytypeid'] = properties['storytypeid'].fillna(0) 
properties['storytypeid'] = properties['storytypeid'].replace(7,1)

# Only has like a couple thousand non NA values, so not worth
properties = properties.drop('typeconstructiontypeid', axis=1)

unitMode = properties['unitcnt'].value_counts().argmax()
properties['unitcnt'] = properties['unitcnt'].fillna(unitMode)

yardSqFt17 = properties['yardbuildingsqft17'].mean()
properties['yardbuildingsqft17'] = properties['yardbuildingsqft17'].fillna(yardSqFt17)

yardSqFt26 = properties['yardbuildingsqft26'].mean()
properties['yardbuildingsqft26'] = properties['yardbuildingsqft26'].fillna(yardSqFt26)

yearBuilt = properties['yearbuilt'].mean()
properties['yearbuilt'] = properties['yearbuilt'].fillna(yearBuilt)

properties['numberofstories'] = properties['numberofstories'].fillna(0)

# Fireplace count already does this
properties = properties.drop('fireplaceflag', axis=1)

structureTax = properties['structuretaxvaluedollarcnt'].mean()
properties['structuretaxvaluedollarcnt'] = properties['structuretaxvaluedollarcnt'].fillna(structureTax)

landTax = properties['landtaxvaluedollarcnt'].mean()
properties['landtaxvaluedollarcnt'] = properties['landtaxvaluedollarcnt'].fillna(landTax)

tax = properties['taxamount'].mean()
properties['taxamount'] = properties['taxamount'].fillna(tax)

# Tax amount already does this
properties = properties.drop('taxvaluedollarcnt', axis=1)

# Idk, I don't wanna deal with these
properties = properties.drop('assessmentyear', axis=1)
properties = properties.drop('taxdelinquencyflag', axis=1)
properties = properties.drop('taxdelinquencyyear', axis=1)

In [107]:
properties.columns

Index([u'parcelid', u'airconditioningtypeid', u'architecturalstyletypeid',
       u'basementsqft', u'bathroomcnt', u'bedroomcnt', u'buildingclasstypeid',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'decktypeid',
       u'finishedfloor1squarefeet', u'calculatedfinishedsquarefeet',
       u'finishedsquarefeet12', u'finishedsquarefeet13',
       u'finishedsquarefeet15', u'finishedsquarefeet50',
       u'finishedsquarefeet6', u'fips', u'fireplacecnt', u'fullbathcnt',
       u'garagecarcnt', u'garagetotalsqft', u'hashottuborspa',
       u'heatingorsystemtypeid', u'lotsizesquarefeet', u'poolcnt',
       u'poolsizesum', u'regionidcounty', u'storytypeid', u'unitcnt',
       u'yardbuildingsqft17', u'yardbuildingsqft26', u'yearbuilt',
       u'numberofstories', u'structuretaxvaluedollarcnt',
       u'landtaxvaluedollarcnt', u'taxamount'],
      dtype='object')

In [108]:
cleanedPropertyData = pd.np.array(properties)
cleanedPropertyData.shape

(2985217, 37)

In [109]:
cleanedPropertyData.nbytes

883624232

In [110]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 37 columns):
parcelid                        int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottub

In [5]:
import csv
predictions = []
t = int(4.5)
y =9.4
predictions.append([t, y,y,y,y,y,y])
predictions.append([t, y,y,y,y,y,y])

firstRow = [['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712']]
with open("temp.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(firstRow)
    writer.writerows(predictions)

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_boston
data, targets = load_boston(True)

allModels = {}

model = linear_model.BayesianRidge()
predicted = cross_val_score(model, data, targets, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

model = linear_model.LinearRegression()
predicted = cross_val_score(model, data, targets, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

In [25]:
allModels

{BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
        fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
        normalize=False, tol=0.001, verbose=False): -3.926780377703075,
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False): -4.0103527020599952}